### API Yahoo Finance

In [73]:
# previamente se  instala las siguientes librerias "yfinance".

In [74]:
# librería para descargar con yahoo finance
import yfinance as yf
# librería para manipular dataframes
import pandas as pd
# librería para manipular fechas
import datetime as dt

In [75]:
# Diccionario con las empresas de las que se traen los datos
dict_Empresas = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Meta": "META",
    "Tesla": "TSLA",
    "Amazon" : "AMZN",

    # Algunas sugerencias:
    
    #"Alphabet": "GOOGL",
    #"JPMorgan": "JPM",
    #"Johnson & Johnson": "JNJ",
    #"Walmart": "WMT"
    }

In [76]:
import os

# Crear el directorio Datasets si no existe
if not os.path.exists('Datasets'):
    os.makedirs('Datasets')

# Bucle para descargar y guardar datos
for key, value in dict_Empresas.items():
    # Se descargan los ultimos 5 años.
    Dataset = yf.download(value, start=dt.datetime.now() - dt.timedelta(days=5*365), end=dt.date.today())
    
    Dataset = Dataset[["Close", "Volume"]]
    # Dataset["Ticker"] = value

    # Se guardan en un archivo con el nombre de la empresa
    Dataset.to_csv(f"Datasets/{key}.csv")

    # Se imprime en pantalla un mensaje de confirmación
    print(f"Archivo {key} cargado exitosamente.")


[*********************100%%**********************]  1 of 1 completed


Archivo Apple cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Microsoft cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Meta cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed


Archivo Tesla cargado exitosamente.


[*********************100%%**********************]  1 of 1 completed

Archivo Amazon cargado exitosamente.


### SQL

In [77]:
import sqlite3

# Conexión a la base de datos
conn = sqlite3.connect('./Database/test.db')
conn.close()

In [78]:
Dataset
#Que es?

,Close,Volume
Date,,
2019-05-28,91.821503,64000000
2019-05-29,90.959503,85580000
2019-05-30,90.816002,62938000
2019-05-31,88.753502,92376000
2019-06-03,84.634499,181974000
...,...,...
2024-05-20,183.539993,30511800
2024-05-21,183.149994,50839100
2024-05-22,183.130005,28148800


In [79]:
import os

# Carpeta con los archivos CSV
folder_path = './Datasets'

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

files

['Amazon.csv', 'Apple.csv', 'Tesla.csv', 'Meta.csv', 'Microsoft.csv']

In [80]:
# Lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# Leer cada archivo CSV en un DataFrame y agregarlo a la lista
for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df['Empresa'] = file.split('.')[0]
        dfs.append(df)

# Concatenar los DataFrames en uno solo
result = pd.concat(dfs, ignore_index=True)

# Imprimir el resultado
result.head()

,Date,Close,Volume,Empresa
0,2019-05-28,91.821503,64000000,Amazon
1,2019-05-29,90.959503,85580000,Amazon
2,2019-05-30,90.816002,62938000,Amazon
3,2019-05-31,88.753502,92376000,Amazon
4,2019-06-03,84.634499,181974000,Amazon


In [81]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
conn.close()

Connected to SQLite


In [82]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='append')
conn.close()

Connected to SQLite


In [83]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='replace')

Connected to SQLite


6295

In [84]:
# Creo una función que haga todo:

def write_sql_main(DB, df, table):
	"""Esta funcion inserta masivamente un df o una lista de tuplas en SQL."""
	try:
		conn = sqlite3.connect(DB)
		print("Connected to SQLite")
		df.to_sql(table, conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
		result = 'ok'
	except sqlite3.Error as error:
		print("Failed to insert multiple records into sqlite table", error)
		result = 'failed'
	finally:
		if conn:
			conn.close()
			print("The SQLite connection is closed")
	return result

In [85]:
write_sql_main(DB=DB, df = result, table = 'main2')

Connected to SQLite
The SQLite connection is closed


'ok'